In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from dateutil import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

In [2]:
conexao = Connect.connect_techdb()
basebtg = Connect.import_table(conexao, "base_btg")
conexao.close()

In [3]:
def import_posicoes():
    conexao = Connect.connect_techdb()
    posicoes = Connect.import_table(conexao, "posicao")
    conexao.close()
    #posicoes = posicoes[posicoes['Estratégia'] != "Pós-fixado"]
    #posicoes = posicoes[['Produto', 'Ativo', "Nome"]]
    #posicoes = posicoes.drop_duplicates("Ativo", keep='last')
    
    return posicoes

posicoes = import_posicoes()

#### Proventos RV

In [4]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.get("https://statusinvest.com.br/acoes/proventos/ibovespa")

In [5]:
data_list = [data.text.replace("R$ ", "").replace("%", "").replace(" ","\n").split("\n") for data in driver.find_elements(By.XPATH, "(//tr)")]
data_list = [data for data in data_list if data != ['']]

In [6]:
proventos_rv = \
    pd.DataFrame(data={
        "ATIVO": [x[0] for x in data_list],
        "VALOR": [x[1] for x in data_list],
        "DATA COM": [x[2] for x in data_list],
        "DATA PAGAMENTO": [x[3] for x in data_list],
        "TIPO": [x[4] for x in data_list],
        "DY": [x[5] for x in data_list],
    })
proventos_rv = proventos_rv.iloc[1:,:]

In [7]:
proventos_rv.rename(columns={"ATIVO":"Ativo", "VALOR":"Valor",
                             "DATA PAGAMENTO":"Data de Pagamento", 
                             "TIPO":"Tipo"}, inplace=True)

In [8]:
proventos_rv['DATA COM'] = pd.to_datetime(proventos_rv['DATA COM'], format="%d/%m/%Y")
proventos_rv['Data de Pagamento'] = pd.to_datetime(proventos_rv['Data de Pagamento'], format="%d/%m/%Y")
proventos_rv['Valor'] = [float(val.replace(",", ".")) for val in proventos_rv['Valor']]

In [11]:
posicoes_rv = posicoes.loc[:,["Conta", "Ativo", "Quantidade", "Valor Bruto"]]

proventos_futuros_rv = proventos_rv.loc[:,["Ativo", "Valor", "Data de Pagamento", "Tipo"]].merge(posicoes_rv, on='Ativo', how='right')
proventos_futuros_rv.dropna(subset='Valor', inplace=True)
proventos_futuros_rv['Provento'] = proventos_futuros_rv['Valor'] * proventos_futuros_rv['Quantidade']
proventos_futuros_rv['Mes do Evento'] = [pd.to_datetime(data).strftime("%Y-%m") for data in proventos_futuros_rv['Data de Pagamento']]

proventos_futuros_rv = proventos_futuros_rv.merge(basebtg.loc[:,["Conta", "Assessor"]], on='Conta', how='left')
proventos_futuros_rv.to_excel(r"C:\Scripts\script_proventos_futuros\proventos_futuros_rf.xlsx", header=True, index=False)

##### Proventos RF - Anbima

In [ ]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.get("https://data.anbima.com.br/datasets/titulos-privados-agenda-de-eventos")

In [ ]:
dfs = []

In [ ]:
while True:
    tipo =  [tipo_titulo.text for tipo_titulo in driver.find_elements(By.XPATH, "//tr//td[1]")]
    codigo = [codigo_produto.text for codigo_produto in driver.find_elements(By.XPATH, "//tr//td[2]")]
    data_do_evento = [data_prod.text for data_prod in driver.find_elements(By.XPATH, "//tr//td[3]")]
    data_de_liquidacao = [data_liq.text for data_liq in driver.find_elements(By.XPATH, "//tr//td[4]")]
    evento = [evento_prod.text for evento_prod in driver.find_elements(By.XPATH, "//tr//td[5]")]
    percentual_pago = [float(percent_prod.text.replace(",", ".").replace(" %", "")) for percent_prod in driver.find_elements(By.XPATH, "//tr//td[6]")]
        
    dfs.append(
        pd.DataFrame(data={
            "Tipo":tipo,
            "Código":codigo,
            "Data do Evento":data_do_evento,
            "Data de Liquidação":data_de_liquidacao,
            "Evento":evento,
            "% Pago":percentual_pago
        })
    )    
        
    click_next = driver.find_element(By.XPATH, '//a[@id="pagination-next-button"]')
    click_next.click()
    time.sleep(3)

In [ ]:
proventos_rf = pd.concat(dfs)
proventos_rf['Data do Evento'] = pd.to_datetime(proventos_rf['Data do Evento'])
proventos_rf['Data de Liquidação'] = pd.to_datetime(proventos_rf['Data de Liquidação'])
proventos_rf = proventos_rf[proventos_rf['Data do Evento'] >= datetime.today() - relativedelta.relativedelta(days=1)]
proventos_rf = proventos_rf.drop_duplicates()
proventos_rf.rename(columns={"Código":"Ativo"}, inplace=True)

In [ ]:
posicoes_rf = \
    posicoes[
        (posicoes['Produto'] == "CRA") |
        (posicoes['Produto'] == "CRI") |
        (posicoes['Produto'] == "DEBENTURE")
    ]
posicoes_rf['Ativo'] = [ativo.split("-")[1] for ativo in posicoes_rf['Ativo']]
posicoes_rf = posicoes_rf.loc[:,["Conta", "Emissor", "Ativo", "Valor Bruto"]]

In [ ]:
proventos_futuros_rf = proventos_rf.loc[:,["Tipo", "Ativo", "Data do Evento", "Evento", "% Pago"]].merge(posicoes_rf, on='Ativo', how='right')
proventos_futuros_rf.dropna(subset=["Tipo"], inplace=True)
proventos_futuros_rf['Provento'] = proventos_futuros_rf['Valor Bruto'] * (proventos_futuros_rf['% Pago'] / 100)
proventos_futuros_rf['Data do Evento'] = [pd.to_datetime(data).strftime("%Y-%m-%d") for data in proventos_futuros_rf['Data do Evento']]
proventos_futuros_rf['Mes do Evento'] = [pd.to_datetime(data).strftime("%Y-%m") for data in proventos_futuros_rf['Data do Evento']]

#### Proventos RF - Admin

In [4]:
proventos_futuros_rf = pd.read_excel(r"C:\Scripts\script_proventos_futuros\Proventos Futuros - Detalhado.xlsx", header=2)
proventos_futuros_rf.drop("Data simulação", axis=1, inplace=True)
proventos_futuros_rf['Conta'] = proventos_futuros_rf['Conta'].astype(str)
proventos_futuros_rf['Mes'] = [pd.to_datetime(data).strftime("%Y-%m") for data in proventos_futuros_rf['Data pagamento']]

In [5]:
proventos_futuros_rf = proventos_futuros_rf.merge(basebtg.loc[:,["Conta", "Assessor"]], on='Conta', how='left')

In [6]:
proventos_futuros_rf.to_excel(r"C:\Scripts\backups_atria\diarios\arquivos_banco\scraping_calculadoraB3\proventos_futuros_rf.xlsx", index=False, header=True)

#### Subir no DB

In [7]:
conexao = Connect.connect_techdb()
proventos_futuros_rf.to_sql("proventos_futuros_rf", 
                            con=conexao, 
                            if_exists='replace', 
                            index=False, schema='principal')
conexao.close()